In [1]:
"""
Test Script

"""
import random
import copy
import numpy as np
from vwgym import VacuumWorld, Vectorise, StepWrapper
from vwgym.fun_lite import *
from torch.distributions import Categorical

import pyworld.toolkit.tools.visutils.jupyter as J
import pyworld.toolkit.tools.gymutils as gu

random.seed(123518)

if torch.cuda.is_available():
    print('GPU Available:\t', True)
    device = 'cuda'
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

else:
    device = 'cpu'

GPU Available:	 True


In [2]:
model_path = 'vwgym/saved_model/vwgym_f_net_step_500000.pt'
checkpoint = torch.load(model_path)

In [3]:
args = checkpoint['args']

In [4]:
env_n = VacuumWorld(args['grid_size'])
env_v = Vectorise(copy.deepcopy(env_n))
env = StepWrapper(env_v)

In [5]:
num_actions = env.action_space.n
input_dim = (3, 8, 8)
f_net = FuNet(input_dim, args['d'], args['len_hist'], args['eps'], args['k'], num_actions, device)
optimizer = torch.optim.Adam(f_net.parameters(), lr=args['lr'], eps=1e-5)

f_net.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optim'])

In [6]:
with torch.no_grad(): # let pytorch know that no gradient should be computed
    f_net.eval() # let the model know that it in test mode, i.e. no gradient and no dropout
    predictions = []
    
    goal_history, s_Mt_hist, ep_binary = f_net.agent_model_init()
    prev_x = env.reset()
    print(prev_x)
    x = torch.from_numpy(prev_x).float()
    step = 0
    prev_action = []
    
    for __ in range(1500):
        action_probs, v_Mt, v_Wt, goal_history, s_Mt_hist = f_net(x, goal_history, s_Mt_hist)
#         a_t = np.argmax(action_probs.to('cpu').numpy()[0])
#         prev_action.append(a_t)
#         if prev_action[-5:].count(a_t) == 5:
        a_t, _, _ = take_action(action_probs)
#                 print('rs')
#                 a_t = a_t[0]
#         if len(prev_action) > 10:
#             prev_action = []
# #         print(np.argmax(action_probs.to('cpu').numpy()[0]))

#         dist = Categorical(action_probs)
#         print(dist.probs)
#         action = dist.sample()
#         print(action)
#         logp = dist.log_prob(action)
#         entropy = dist.entropy()
        # print('previous:\n', prev_x)
#         x, reward, done, ep_info = env.step(prev_x, a_t)
        x, reward, done, ep_info = env.step(prev_x, a_t[0])
        
        # print('Done\t:', done)
        # print('action:\t', env.action_meanings[a_t])
        # print('post:\n', x)
        # print('-'*50)
        if done:
            print(f"total steps = {step} | reward = {ep_info['ep_rewards']} | length = {ep_info['ep_len']}")
            print(x)
            break
        else:
            prev_x = x
        x = torch.from_numpy(x).float()
        ep = torch.FloatTensor([1.0 - done]).to(device)
        # print('EpisodeStatus:\t', ep)
        ep_binary.pop(0)
        ep_binary.append(ep)
        
#         predictions.append(a_t)
        predictions.append(a_t[0])        

[[[  0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0 255   0]
  [  0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0]]

 [[  0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0 128   0]
  [  0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0]]

 [[  0   0   0   0   0 128   0 128]
  [  0   0   0   0   0   0   0   0]
  [255   0   0 255 255   0   0   0]
  [  0   0   0   0   0 255 128   0]
  [128   0   0   0   0   0   0   0]
  [255   0   0   0 128 128 255   0]
  [  0   0   0 128 128   0 255   0]
  [255   0   0   0   0   0   0   0]]]
{'ep_rewards': -1094, 'ep_len': 1500, 'move': 293, 'clean': 308, 'turn_left': 325, 'turn_right': 573, 'idle': 1}
total steps = 0 |

In [7]:
import pandas as pd

In [8]:
p = pd.DataFrame.from_dict({'actions': predictions, 'action_meanings':[env.action_meanings[i] for i in predictions]})
p['action_meanings'].value_counts().reset_index()

,index,action_meanings
0,turn_right,572
1,turn_left,325
2,clean,308
3,move,293
4,idle,1


In [9]:
print(len(predictions))
predictions.append(4)
print(len(predictions))
print(env_v.reset())

1499
1500
[[[  0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0 255   0]
  [  0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0]]

 [[  0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0 128   0]
  [  0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0]]

 [[  0   0   0   0   0 128   0 128]
  [  0   0   0   0   0   0   0   0]
  [255   0   0 255 255   0   0   0]
  [  0   0   0   0   0 255 128   0]
  [128   0   0   0   0   0   0   0]
  [255   0   0   0 128 128 255   0]
  [  0   0   0 128 128   0 255   0]
  [255   0   0   0   0   0   0   0]]]


In [10]:
ac_1 = iter(predictions)
ac_2 = iter(predictions)

nstate = gu.episode(env_n, lambda _: next(ac_1), max_length=len(predictions))[0]
vstate = gu.episode(env_v, lambda _: next(ac_2), max_length=len(predictions))[0]

vis = np.concatenate([vstate[:,None,i] for i in [0,1,2]], axis=3)

J.images(vis, scale=40, on_interact=list(zip(np.array(env.action_meanings)[predictions],nstate)));

interactive(children=(IntSlider(value=0, description='x', layout=Layout(width='99%'), max=1499), Output()), _d…

In [11]:
# torch.dot(torch.tensor([2, 3]), torch.tensor([2, 1]))

# a, b = torch.rand(1, 8), torch.rand(8, 5)

# nn.functional.softmax(torch.mm(a, b))

# import time

# time.sleep()

In [12]:
torch.argmax(torch.rand(1, 5))

tensor(4)